In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import random
import string
import hashlib
import json

#Loading original data
columns_name = ['id', 'date', 'latitude', 'longitude' ]

df = pd.read_csv('real_data_set.csv', names = columns_name, sep = '\t')
df['date'] = df['date'].str.slice(stop = 16)
df['date'] = pd.to_datetime(df['date'], format = "%Y-%m-%d %H:%M")
df['week'] = df['date'].dt.isocalendar().week

In [2]:
anon = pd.read_csv('C:/Users/Phuoc Le/Data_Anom/StrawHats_567', names = columns_name, sep = '\t')
anon = anon.loc[anon.id != 'DEL']
anon['date'] = anon['date'].str.slice(stop = 16)
anon['date'] = pd.to_datetime(anon['date'], format = "%Y-%m-%d %H:%M")
anon['week'] = anon['date'].dt.isocalendar().week
anon.latitude = anon.latitude.astype(float)
anon.longitude = anon.longitude.astype(float)

In [3]:
print(df)

           id                date  latitude  longitude  week
0           1 2015-03-04 00:35:00  4.870147  45.772140    10
1           1 2015-03-04 00:35:00  4.870218  45.772095    10
2           1 2015-03-04 00:35:00  4.870210  45.772072    10
3           1 2015-03-04 00:35:00  4.870210  45.772072    10
4           1 2015-03-04 00:35:00  4.870210  45.772072    10
...       ...                 ...       ...        ...   ...
34551844  110 2015-03-12 16:23:00  2.343094  48.891650    11
34551845  110 2015-03-12 16:23:00  2.343094  48.891650    11
34551846  110 2015-03-12 16:23:00  2.343094  48.891649    11
34551847  110 2015-03-12 16:23:00  2.343094  48.891649    11
34551848  110 2015-03-12 19:29:00  2.343127  48.891772    11

[34551849 rows x 5 columns]


In [4]:
print(anon)

                  id                date  latitude  longitude  week
0         uzHUVUEcYt 2015-03-26 18:00:00     4.926     45.780    13
1         iDzWYzS1tF 2015-04-06 17:00:00     4.848     45.763    15
2         GnVEBWXtkp 2015-04-04 21:00:00     6.724     43.491    14
4         hMZp6HFzm3 2015-03-28 20:00:00     4.657     46.076    13
5         iDzWYzS1tF 2015-05-05 01:00:00     4.872     45.785    19
...              ...                 ...       ...        ...   ...
34551844  e37Ww3e4Ve 2015-04-26 00:00:00     4.866     45.756    17
34551845  gleUS0p8a4 2015-03-04 16:00:00     6.890     45.982    10
34551846  gleUS0p8a4 2015-05-04 18:00:00     4.860     45.766    19
34551847  e37Ww3e4Ve 2015-03-28 19:00:00     4.866     45.756    13
34551848  gleUS0p8a4 2015-03-19 02:00:00     4.756     45.995    12

[31096665 rows x 5 columns]


In [5]:
#List of id origin, id anonym, group of id-week

id_week_ori = df.groupby('id')['week'].unique()
id_week_anon = anon.groupby('id')['week'].unique()

list_id_ori = df["id"].unique()
list_id_anon = anon["id"].unique()

print(np.sort(list_id_ori), len(list_id_ori))
print(np.sort(list_id_anon), len(list_id_anon))

[  1   2   4   5   6   7   8   9  11  13  14  15  16  17  18  21  23  24
  25  26  27  28  29  30  31  32  34  35  36  37  38  39  41  42  43  44
  48  49  50  51  52  53  54  55  58  59  60  62  63  65  66  67  68  69
  70  71  72  73  75  77  78  81  83  84  87  89  98 107 110] 69
['0kDIZVKSRa' '1vJYuhstiO' '3via5Ssa3p' '4BVdoRYzRx' '4EyXWmsXDh'
 '4vysrWVMuv' '6EXdHmhzjO' '7ycgA4z323' '88xM2PDuLL' '9TFUM7RXXi'
 'AFHYwL9ucf' 'ALB3t56j9f' 'AgCo6tWszi' 'BfnxnqNCN2' 'EAGWofXQV1'
 'FnOKxVPYIm' 'GGOVxTcJ5N' 'GnVEBWXtkp' 'HAs69lN5vA' 'IRJkz5sT3l'
 'NJNm5RBXTW' 'Nx9HJlxOsU' 'RxoygY3OBB' 'SagxZxR4qz' 'TPkBQxVX14'
 'TZIAXuV0gb' 'UCc1opqDC7' 'UkXn2kxKky' 'VaEKLlIfwE' 'VoafUILy3H'
 'XZjyHOiwic' 'XgFEXY27SH' 'XkLzunVM4l' 'Y3CGYhGuCd' 'YmPCWu7bmd'
 'ccCzj5Ylu2' 'cmtu4hJPzO' 'dMVq2wnfEd' 'e37Ww3e4Ve' 'ekoyXxrInW'
 'f44Qv53OFH' 'gE5QiPdnGP' 'gGpiPwj1fX' 'gleUS0p8a4' 'h7EEAWjYqu'
 'hMZp6HFzm3' 'hkPslAJYPD' 'iDzWYzS1tF' 'jMEJGCWYMV' 'k9s2nTNLh6'
 'kEfMVNFrRG' 'lyrmjlp8t5' 'obPPAwxNhI' 'pdYiTjoFTU' 'qE

In [6]:
#Group id--week--moyen_latitude--moyen_longitude--mean_date--count_line--mean_hour

#Original Table
df['hour'] = df['date'].dt.hour
mean_hour = df.groupby(['id','week'])['hour'].mean()

mean_lat_ori = df.groupby(['id','week'])['latitude'].mean()
mean_long_ori = df.groupby(['id','week'])['longitude'].mean()

mean_date_ori = df.groupby(['id','week'])['date'].mean()
count_ligne_ori = df.groupby(['id','week'])['date'].count()
mean_hour_ori = df.groupby(['id','week'])['hour'].mean()

ori_mean = df[['id','week']].drop_duplicates().reset_index(drop=True)
ori_mean['mean_latitude'] = ori_mean.apply(lambda x: mean_lat_ori[x.id][x.week], axis=1)
ori_mean['mean_longitude'] = ori_mean.apply(lambda x: mean_long_ori[x.id][x.week], axis=1)
ori_mean['mean_date'] = ori_mean.apply(lambda x: mean_date_ori[x.id][x.week], axis=1)
ori_mean['count_line'] = ori_mean.apply(lambda x: count_ligne_ori[x.id][x.week], axis=1)
ori_mean['mean_hour'] = ori_mean.apply(lambda x: mean_hour[x.id][x.week], axis=1)

#Anonymisation Table
anon['hour'] = anon['date'].dt.hour
mean_lat_anon = anon.groupby(['id','week'])['latitude'].mean()
mean_long_anon = anon.groupby(['id','week'])['longitude'].mean()
mean_date_anon = anon.groupby(['id','week'])['date'].mean()
count_ligne_anon = anon.groupby(['id','week'])['date'].count()
mean_hour_anon = anon.groupby(['id','week'])['hour'].mean()

anon_mean = anon[['id','week']].drop_duplicates().reset_index(drop=True)
anon_mean['mean_latitude'] = anon_mean.apply(lambda x: mean_lat_anon[x.id][x.week] , axis=1)
anon_mean['mean_longitude'] = anon_mean.apply(lambda x: mean_long_anon[x.id][x.week], axis=1)
anon_mean['mean_date'] = anon_mean.apply(lambda x: mean_date_anon[x.id][x.week], axis=1)
anon_mean['count_line'] = anon_mean.apply(lambda x: count_ligne_anon[x.id][x.week], axis=1)
anon_mean['mean_hour'] = anon_mean.apply(lambda x: mean_hour_anon[x.id][x.week], axis=1)

In [7]:
ori_mean['new_id'] = ori_mean['id']
scale = anon.shape[0] / df.shape[0]
for i in list_id_ori:
    for w in id_week_ori[i]:
        list_distance = list() 
        mean_lat = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'mean_latitude'].to_list()[0]
        mean_long = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'mean_longitude'].to_list()[0]
        mean_date = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'mean_date'].to_list()[0]
        number_line = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'count_line'].to_list()[0]
        mean_hour = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'mean_hour'].to_list()[0]
        
        group_week = anon_mean.loc[anon_mean.week ==w,['id','week','mean_latitude','mean_longitude','count_line','mean_date','mean_hour']].reset_index(drop=True)
        for j in range(len(group_week)):
            distance = np.power((group_week.mean_latitude[j] - mean_lat)**2 + (group_week.mean_longitude[j] - mean_long)**2,1/2)
            
            delta_ratio = np.power(abs((group_week.count_line[j]/anon.shape[0] - number_line/df.shape[0])),1)
            delta_time = np.power(abs((group_week.mean_date[j] - mean_date).total_seconds() / 3600),1/2)
            
            
            delta_hour = np.power(abs((group_week.mean_hour[j] - mean_hour)),1/2)
            
            list_distance.append(distance*delta_ratio*delta_time*delta_hour)
            
        index = list_distance.index(min(list_distance))
        l = id_week_anon[group_week.id[index]]# list of week have id anon_mean.id[index] 
        while ((w not in l)):
            list_distance[index] = 1000000000
            index = list_distance.index(min(list_distance).all())
            l = id_week_anon[group_week.id[index]]
        if (w in l):
            ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'new_id'] = group_week.id[index]
        
#print(ori_mean.head())
print(ori_mean.new_id.nunique())

result = {}
count2 = 0
for i in list_id_ori:
    result[str(i)] = {}
    for w in id_week_ori[i]:
        result[str(i)]['2015'+'-'+str(w)] = [str(ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'new_id'][count2])]
        count2 +=1
print(result)

69
{'1': {'2015-10': ['z4GASA4OQr'], '2015-11': ['z4GASA4OQr'], '2015-12': ['z4GASA4OQr'], '2015-13': ['z4GASA4OQr'], '2015-14': ['z4GASA4OQr'], '2015-15': ['z4GASA4OQr'], '2015-16': ['z4GASA4OQr'], '2015-17': ['z4GASA4OQr'], '2015-18': ['z4GASA4OQr'], '2015-19': ['z4GASA4OQr'], '2015-20': ['z4GASA4OQr']}, '2': {'2015-10': ['ekoyXxrInW'], '2015-11': ['ekoyXxrInW'], '2015-12': ['ekoyXxrInW'], '2015-13': ['ekoyXxrInW'], '2015-14': ['ekoyXxrInW'], '2015-15': ['ekoyXxrInW'], '2015-16': ['ekoyXxrInW'], '2015-17': ['ekoyXxrInW'], '2015-18': ['ekoyXxrInW'], '2015-19': ['ekoyXxrInW'], '2015-20': ['ekoyXxrInW']}, '107': {'2015-10': ['vfSqUrpIUV'], '2015-11': ['vfSqUrpIUV'], '2015-12': ['vfSqUrpIUV']}, '98': {'2015-10': ['BfnxnqNCN2'], '2015-11': ['BfnxnqNCN2']}, '21': {'2015-10': ['0kDIZVKSRa'], '2015-11': ['0kDIZVKSRa']}, '5': {'2015-10': ['UCc1opqDC7'], '2015-11': ['UCc1opqDC7']}, '9': {'2015-10': ['7ycgA4z323'], '2015-11': ['7ycgA4z323'], '2015-12': ['7ycgA4z323'], '2015-13': ['7ycgA4z323'],

In [8]:
jsonData = json.dumps(result, indent=4)
print(jsonData)

{
    "1": {
        "2015-10": [
            "z4GASA4OQr"
        ],
        "2015-11": [
            "z4GASA4OQr"
        ],
        "2015-12": [
            "z4GASA4OQr"
        ],
        "2015-13": [
            "z4GASA4OQr"
        ],
        "2015-14": [
            "z4GASA4OQr"
        ],
        "2015-15": [
            "z4GASA4OQr"
        ],
        "2015-16": [
            "z4GASA4OQr"
        ],
        "2015-17": [
            "z4GASA4OQr"
        ],
        "2015-18": [
            "z4GASA4OQr"
        ],
        "2015-19": [
            "z4GASA4OQr"
        ],
        "2015-20": [
            "z4GASA4OQr"
        ]
    },
    "2": {
        "2015-10": [
            "ekoyXxrInW"
        ],
        "2015-11": [
            "ekoyXxrInW"
        ],
        "2015-12": [
            "ekoyXxrInW"
        ],
        "2015-13": [
            "ekoyXxrInW"
        ],
        "2015-14": [
            "ekoyXxrInW"
        ],
        "2015-15": [
            "ekoyXxrInW"
        ],
 